# Práctica 3 - Metaheurística

## Funciones globales

In [1]:
import numpy as np
import random
from datetime import datetime

def leer_parametros(nombre_archivo):
  parametros = {}
  with open(nombre_archivo, "r") as f:
    for linea in f:
      linea = linea.strip()
      if linea and "=" in linea:
        clave, valor = linea.split("=")
        parametros[clave.strip()] = valor.strip()
  return parametros

# Funcion para leer el fichero de datos.
def leer_instancia(nombre_archivo):
    with open(nombre_archivo, "r") as f:
        lineas = [linea.strip() for linea in f.readlines() if linea.strip() != ""]

    n = int(lineas[0])

    flujo_lineas = lineas[1 : 1 + n]
    distancia_lineas = lineas[1 + n : 1 + 2*n]

    flujo = np.array([list(map(int, fila.split())) for fila in flujo_lineas])
    distancia = np.array([list(map(int, fila.split())) for fila in distancia_lineas])

    return n, flujo, distancia

# Funcion para hacer el sumatorio por filas de las matrices construidas
def sumatorio(n, flujo, distancia):
  vflujo=np.zeros(n)
  vdist=np.zeros(n)
  permutacion=np.zeros(n)
  for i in range(n):
    permutacion[i]=i+1
    for j in range(n):
      vflujo[i]+=flujo[i][j]
      vdist[i]+=distancia[i][j]
  return vflujo, vdist, permutacion

# Funcion para ordenar de mayor a menor los vectores Distancia y
# Flujo (con la permutación)
def ordenar(diccd, diccf):
  diccf = sorted(diccf.items(), key=lambda item: item[1], reverse=True)
  diccd = sorted(diccd.items(), key=lambda item: item[1])
  return diccd, diccf

def aleatorizar(permutacion, vdist):

  return permutacion

def coste(n, flujo, distancia, permutacionDef):
  coste=0
  for i in range(n):
    for j in range(n):
      if(i!=j):
        coste+=flujo[i][j]*distancia[int(permutacionDef[i])][int(permutacionDef[j])]
  return coste

def factorizacion(n, flujo, distancia, permutacion,r,s):
  if r==s:
    return 0
  loc_r=permutacion[r]
  loc_s=permutacion[s]
  delta=0
  for k in range(len(permutacion)):
      if k != r and k != s:
        loc_k=permutacion[k]
        delta += flujo[r][k] * (distancia[loc_s][loc_k] - distancia[loc_r][loc_k])
        delta += flujo[k][r] * (distancia[loc_k][loc_s] - distancia[loc_k][loc_r])
        delta += flujo[s][k] * (distancia[loc_r][loc_k] - distancia[loc_s][loc_k])
        delta += flujo[k][s] * (distancia[loc_k][loc_r] - distancia[loc_k][loc_s])
  delta += flujo[r][s] * (distancia[loc_s][loc_r] - distancia[loc_r][loc_s])
  delta += flujo[s][r] * (distancia[loc_r][loc_s] - distancia[loc_s][loc_r])

  return delta

def escritura_logs(ruta, contenido, tiempo, param=None, tamano=None):
    if param is None:
        param = ""
    if tamano is None:
        tamano = "No especificado"

    log_text = (
        "Log " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n"
        + "Tamaño del problema: " + str(tamano) + "\n"
        + "Parametros: " + param + "\n"
        + "Costes: " + contenido + "\n"
        + "Tiempo empleado: " + str(tiempo.total_seconds()) + " segundos\n"
        + "-"*50 + "\n"
    )

    with open(ruta, "a") as f:
        f.write(log_text)

    print("\n--- REGISTRO DE EJECUCIÓN ---")
    print(log_text)


## Greedy normal

In [2]:
def greedy1(ruta):

    inicio = datetime.now()
    n, flujo, distancia = leer_instancia(ruta)
    vflujo, vdist, permutacion = sumatorio(n, flujo, distancia)

      # Aqui unicamente creamos una estructura de dupla para almacenar los indices
      # antes de ordenar los vectores, ya que de otra forma perderiamos la
      # permutación.
    diccf={}
    diccd={}
    for i in range(n):
        diccf[i + 1] = float(vflujo[i])
        diccd[i + 1] = float(vdist[i])

      # Ordenamos los vectores
    diccd, diccf = ordenar(diccd, diccf)

      #Construimos Permutación

    permf=np.zeros(n)
    permd=np.zeros(n)
    i = 0
    for clave in diccf:
      permf=clave[0]
      i+=1
    i = 0
    for clave in diccd:
      permd=clave[0]
      i+=1

    print("Matriz de flujo:")
    print(flujo)

    print("Matriz de distancia:")
    print(distancia)
    print()
    print("Vector flujo mayor a menor", diccf)
    print("Vector distancia menor a mayor", diccd)
    permf=[]
    permd=[]
    for i in diccf:
      permf.append(int(i[0]-1))
    for i in diccd:
      permd.append(int(i[0]-1))

    print(permf)
    print(permd)

    permutacionDef=np.zeros(n)
    i=0
    for perm in permf:
      permutacionDef[perm]=permd[i]
      i+=1

    fin = datetime.now()
    duracion = fin - inicio
    print(permutacionDef)

    print()
    print("Coste total: ", coste(n, flujo, distancia, permutacionDef))
        # Parametros log consistentes
    param = f"Algoritmo: greedy - Dataset: {ruta} - Semilla: {parametros.get('seed',0)} - K: {parametros.get('k','')}"
    escritura_logs(
        f"log_greedy{dataset}.txt",
        f"Permutación Greedy: {permutacionDef} -> Coste Greedy: {coste(n, flujo, distancia, permutacionDef)}",
        duracion,
        param,
        tamano=n
    )
    return permutacionDef, coste

## Greedy aleatorio

In [3]:
def greedy_aleatorio(ruta, alg=None, semilla=None):

    inicio=datetime.now()

    try:
        K = int(parametros["k"])
    except:
        raise KeyError("Falta el parámetro 'k' en el fichero de parámetros")

    if semilla is not None:
        random.seed(semilla)

    n, flujo, distancia = leer_instancia(ruta)
    vflujo, vdist, permutacion = sumatorio(n, flujo, distancia)

    # Creamos diccionarios
    diccf = {i+1: vflujo[i] for i in range(n)}
    diccd = {i+1: vdist[i] for i in range(n)}

    # Ordenamos flujo y distancia como en el greedy anterior
    diccd, diccf = ordenar(diccd, diccf)

    # Listas de candidatos
    unidades = [u[0] for u in diccf]
    localizaciones = [d[0] for d in diccd]

    permutacionDef = np.zeros(n, dtype=int)

    # Construcción aleatoria
    for _ in range(n):
        # elegimos aleatoriamente entre los K mejores disponibles
        u = random.choice(unidades[:min(K, len(unidades))])
        l = random.choice(localizaciones[:min(K, len(localizaciones))])

        permutacionDef[u-1] = l-1

        # eliminamos los ya usados
        unidades.remove(u)
        localizaciones.remove(l)

    # Calcular coste
    cost = coste(n, flujo, distancia, permutacionDef)

    fin = datetime.now()
    duracion = fin - inicio

    param = f"Algoritmo: {alg if alg else 'greedy'} - Dataset: {ruta} - Semilla: {semilla} - K: {K}"

    print("Permutación generada (unidad -> localización):", permutacionDef)
    print("Coste total (GRA):", cost)

    if alg == "greedyal":
        escritura_logs(
            f"log_greedy_aleatorio{dataset}{semilla}.txt",
            f"Permutación GRA: {permutacionDef} -> Coste GRA: {cost}",
            duracion,
            param,
            tamano=n
        )
    return permutacionDef, cost

## Algoritmo Evolutivo Generacional (GEN)


In [4]:
# Seleccionamos el mejor individuo entre k elegidos al azar
def torneo(poblacion, costes, k):
    candidatos = random.sample(range(len(poblacion)), k)
    mejor = min(candidatos, key=lambda idx: costes[idx])
    return poblacion[mejor]

# Este operador mezcla dos padres manteniendo el orden de los elementos y evitando duplicados
def cruce_OX2(padre1, padre2):

    n = len(padre1)
    hijo = [-1] * n  # iniciamos el hijo vacío

    # En primer lugar seleccionamos de forma aleatoria varias posiciones del padre1
    posiciones = random.sample(range(n), k=random.randint(2, n // 2))

    # Copiamos esos valores al hijo
    for pos in posiciones:
        hijo[pos] = padre1[pos]

    # Rellenamos con los valores del padre2 en orden
    pos_hijo = 0
    for gen in padre2:
        if gen not in hijo:
            # Buscamos la siguiente posición vacía en el hijo
            while hijo[pos_hijo] != -1:
                pos_hijo += 1
            hijo[pos_hijo] = gen

    return np.array(hijo)

def cruce_MOC(padre1,padre2):
    """
    Esta es una variante del cruce de orden, funciona parecido a OX2 pero en lugar de copiar posiciones sueltas, copia un bloque continuo del primer padre y después completa con los genes del segundo

    """
    n = len(padre1)
    hijo = [-1] * n

    #Elegimos posiciones de inicio y fin del bloque
    i, j = sorted(random.sample(range(n), 2))

    #Copiamos el bloque del padre1
    hijo[i:j+1] = padre1[i:j+1]

    #Rellenamos con los elementos del padre2 en orden
    pos = (j + 1) % n
    for gen in padre2:
        if gen not in hijo:
            hijo[pos] = gen
            pos = (pos + 1) % n

    return np.array(hijo)

def mutacion_2opt(permutacion):
    """
    Intercambia dos posiciones aleatorias para generar un nuevo individuo

    """
    n = len(permutacion)
    i, j = random.sample(range(n), 2)  # elegimos dos posiciones distintas
    nueva_perm = permutacion.copy()

    # Intercambiamos las posiciones
    nueva_perm[i], nueva_perm[j] = nueva_perm[j], nueva_perm[i]

    return np.array(nueva_perm)

def algoritmo_gen(ruta, usar_parametros="parametros.txt"):
    inicio = datetime.now()
    params = leer_parametros(usar_parametros)

    try:
        tipo_cruce = params["cruce"] # Tipo de operador de cruce que usaremos
        M = int(params["M"]) # Tamaño de la población
        E = int(params["E"]) # Número de mejores individuos que pasan directamente sin modificarse
        kBest = int(params["kBest"]) # Nº de individuos que compiten para elegir los padres
        kWorst = int(params["kWorst"]) # Nº de individuos que compiten para eliminar los peores
        max_eval = int(params["max_eval"]) # Número máximo de evaluaciones
        prob_cruce = float(params["prob_cruce"]) # Probabilidad de aplicar el cruce entre dos padres
        prob_mut = float(params["prob_mut"]) # Probabilidad de aplicar la mutación 2-opt a un individuo
        semilla = int(params["seed"]) # Semilla aleatoria para que los resultados sean reproducibles
        k = int(params["k"]) # Parámetro k para el greedy aleatorio
        max_time = int(params["max_time"]) # Tiempo máximo de ejecución
    except KeyError as e:
        raise KeyError(f"Falta el parámetro requerido en el fichero: {e}")

    random.seed(semilla)
    np.random.seed(semilla)

    n, flujo, distancia = leer_instancia(ruta)

    poblacion = []
    costes = []

    for i in range(M):
        if i < int(M * 0.2):
            perm, cost = greedy_aleatorio(ruta, k)
        else:
            perm = np.random.permutation(n)
            cost = coste(n, flujo, distancia, perm)
        poblacion.append(perm)
        costes.append(cost)

    poblacion = np.array(poblacion)
    costes = np.array(costes)

    evaluaciones = M
    mejor_global = poblacion[np.argmin(costes)]
    mejor_coste = np.min(costes)

    while evaluaciones < max_eval and (datetime.now() - inicio).total_seconds() < max_time: ## añadir a los parametros. ##
        nueva_poblacion = []
        nueva_costes = []

        # Elitismo: copiamos los E mejores individuos
        elite_idx = np.argsort(costes)[:E]
        elite = [poblacion[i].copy() for i in elite_idx]
        elite_costes = [costes[i] for i in elite_idx]

        # Generamos nueva población
        while len(nueva_poblacion) < M:
            padres_idx = random.sample(range(M),2*kBest)
            torneo1 = padres_idx[:kBest]
            torneo2 = padres_idx[kBest:]

            padres1 = poblacion[min(torneo1,key=lambda x: costes[x])]
            padres2 = poblacion[min(torneo2,key=lambda x: costes[x])]

            # Cruce
            if random.random() < prob_cruce:
                if tipo_cruce.upper() == "OX2":
                    hijo = cruce_OX2(padres1, padres2)
                else:
                    hijo = cruce_MOC(padres1, padres2)
            else:
                hijo = padres1.copy()

            # Mutación
            if random.random() < prob_mut:
                hijo = mutacion_2opt(hijo)

            cost_hijo = coste(n, flujo, distancia, hijo)
            nueva_poblacion.append(hijo)
            nueva_costes.append(cost_hijo)
            evaluaciones += 1

            if evaluaciones >= max_eval:
                break

        # Reemplazo con elitismo
        nueva_poblacion = np.array(nueva_poblacion)
        nueva_costes = np.array(nueva_costes)

        for e in range(E):
    # Seleccionamos kWorst individuos aleatoriamente
            perdedores = random.sample(range(M), kWorst)
            peor_idx = max(perdedores, key=lambda x: nueva_costes[x])
            nueva_poblacion[peor_idx] = elite[e]
            nueva_costes[peor_idx] = elite_costes[e]

        poblacion = nueva_poblacion
        costes = nueva_costes

        if np.min(costes) < mejor_coste:
            mejor_coste = np.min(costes)
            mejor_global = poblacion[np.argmin(costes)]

    fin = datetime.now()
    duracion = fin - inicio

    print(f"--- RESULTADOS GENERACIONAL ---")
    print(f"Dataset: {ruta}")
    print(f"Cruce: {tipo_cruce}, E={E}, kBest={kBest}, kWorst={kWorst}")
    print(f"Mejor coste encontrado: {mejor_coste}")
    print(f"Tiempo total: {duracion}")

    escritura_logs(
        f"log_gen_{tipo_cruce}_{semilla}.txt",
        f"Mejor coste GEN-{tipo_cruce}: {mejor_coste}",
        duracion,
        f"Algoritmo: GEN - Dataset: {ruta} - Cruce: {tipo_cruce} - E={E}, kBest={kBest}, kWorst={kWorst}, prob_cruce={prob_cruce}, prob_mut={prob_mut}, M={M}, K={k}, max_eval={max_eval}, Semilla={semilla}",
        tamano=n
    )

    return mejor_global, mejor_coste, duracion

# Algoritmo tabú

In [5]:
from collections import deque

def busqueda_tabu_modificada(permutacion_inicial, coste_inicial, max_iter, n, flujo, distancia, tenencia_tabu):
    """
    - Búsqueda Tabú Modificada.
    - Sin memoria a largo plazo ni oscilación.
    """
    mejor_perm_local = permutacion_inicial.copy()
    mejor_coste_local = coste_inicial

    actual_perm = permutacion_inicial.copy()
    actual_coste = coste_inicial

    tabu_list = deque(maxlen=tenencia_tabu)

    # Inicialización del vector DLB
    dlb = [0] * n

    iteracion = 0
    while iteracion < max_iter:
        mejor_mov = None
        mejor_delta = float('inf')

        encontrado_movimiento = False

        # Exploración del vecindario con DLB
        for i in range(n):
            if dlb[i] == 1:
                continue

            mejora_con_i = False

            for j in range(i + 1, n):
                delta = factorizacion(n, flujo, distancia, actual_perm, i, j)
                coste_vecino = actual_coste + delta

                movimiento = (i, j)
                es_tabu = (movimiento in tabu_list) or ((j, i) in tabu_list)
                aspiracion = es_tabu and (coste_vecino < mejor_coste_local)

                if not es_tabu or aspiracion:
                    if delta < mejor_delta:
                        mejor_delta = delta
                        mejor_mov = movimiento
                        encontrado_movimiento = True

                    if delta < 0:
                        mejora_con_i = True

            if not mejora_con_i:
                dlb[i] = 1

        # Gestión de estancamiento por DLB
        # Si todos los bits están a 1 o no encontramos nada, reseteamos el DLB
        if mejor_mov is None:
            if all(bit == 1 for bit in dlb):
                dlb = [0] * n
                continue
            else:
                break

        # Aplicar Movimiento
        i, j = mejor_mov
        actual_perm[i], actual_perm[j] = actual_perm[j], actual_perm[i]
        actual_coste += mejor_delta

        # Actualizar Tabú
        tabu_list.append((i, j))

        # Actualizar DLB
        dlb[i] = 0
        dlb[j] = 0

        # Actualizar Mejor Global
        if actual_coste < mejor_coste_local:
            mejor_coste_local = actual_coste
            mejor_perm_local = actual_perm.copy()

        iteracion += 1

    return mejor_perm_local, mejor_coste_local

# Memetico generacional

In [8]:
def algoritmo_memetico_gen(ruta, usar_parametros="parametros.txt"):
    inicio = datetime.now()
    # Leemos parámetros locales para configurar el memético
    params = leer_parametros(usar_parametros)

    try:
        # Fijar parámetros
        E = 1
        kBest = 2
        prob_cruce = 0.70
        max_eval = 10000

        # Parámetros del fichero
        M = int(params["M"])
        prob_mut = float(params["prob_mut"])
        semilla = int(params["seed"])
        k_greedy = int(params["k"])
        tenencia_tabu = int(params["tenenciatabu"])
        kWorst = int(params.get("kWorst", 2))

    except KeyError as e:
        raise KeyError(f"Falta el parámetro requerido en el fichero: {e}")

    # Fijar semillas
    random.seed(semilla)
    np.random.seed(semilla)

    n, flujo, distancia = leer_instancia(ruta)

    # Inicialización de la Población
    poblacion = []
    costes = []

    for i in range(M):
        if i < int(M * 0.2):
            perm, cost = greedy_aleatorio(ruta, "", semilla)
        else:
            perm = np.random.permutation(n)
            cost = coste(n, flujo, distancia, perm)

        poblacion.append(perm)
        costes.append(cost)

    poblacion = np.array(poblacion)
    costes = np.array(costes)

    evaluaciones = M

    # Mejor Global Histórico
    idx_mejor_init = np.argmin(costes)
    mejor_global = poblacion[idx_mejor_init].copy()
    mejor_coste = costes[idx_mejor_init]

    # Control para triggers de BT
    ultimo_trigger = 0

    print(f"Inicio MGEN. Mejor coste inicial: {mejor_coste}")

    # Bucle Generacional
    while evaluaciones < max_eval:

        nueva_poblacion = []
        nueva_costes = []

        # Elitismo (E=1)
        idx_elite = np.argmin(costes)
        elite_individuo = poblacion[idx_elite].copy()
        elite_coste = costes[idx_elite]

        # Generación de descendientes
        while len(nueva_poblacion) < M:
            # Selección: Torneo Binario (kBest=2)
            candidatos1 = random.sample(range(M), kBest)
            idx_padre1 = min(candidatos1, key=lambda x: costes[x])

            candidatos2 = random.sample(range(M), kBest)
            idx_padre2 = min(candidatos2, key=lambda x: costes[x])

            padre1 = poblacion[idx_padre1]
            padre2 = poblacion[idx_padre2]

            # Cruce OX2 (70%)
            if random.random() < prob_cruce:
                hijo = cruce_OX2(padre1, padre2)
            else:
                hijo = padre1.copy()

            # Mutación
            if random.random() < prob_mut:
                hijo = mutacion_2opt(hijo)

            # Evaluación
            cost_hijo = coste(n, flujo, distancia, hijo)
            nueva_poblacion.append(hijo)
            nueva_costes.append(cost_hijo)
            evaluaciones += 1

            if evaluaciones >= max_eval:
                break

        nueva_poblacion = np.array(nueva_poblacion)
        nueva_costes = np.array(nueva_costes)

        # Reemplazo de los peores con el Élite
        if len(nueva_poblacion) == M:
            candidatos_reemplazo = random.sample(range(M), kWorst)
            idx_peor = max(candidatos_reemplazo, key=lambda x: nueva_costes[x])

            nueva_poblacion[idx_peor] = elite_individuo
            nueva_costes[idx_peor] = elite_coste

        poblacion = nueva_poblacion
        costes = nueva_costes

        # Aplicación de Búsqueda Tabú (Memético)
        profundidad = 0
        hito_actual = 0

        # Lógica de triggers (1000, 2000, 5000)
        k_evals = evaluaciones

        # Añadir el k_evals y ultimo_trigger en parametros !!!!
        if (k_evals // 5000) > (ultimo_trigger // 5000):
            profundidad = 100
            hito_actual = 5000
        elif (k_evals // 2000) > (ultimo_trigger // 2000):
            profundidad = 50
            hito_actual = 2000
        elif (k_evals // 1000) > (ultimo_trigger // 1000):
            profundidad = 10
            hito_actual = 1000

        if profundidad > 0:
            # Aplicar sobre el ELITE actual
            idx_mejor_actual = np.argmin(costes)
            perm_elite_bl = poblacion[idx_mejor_actual].copy()
            coste_elite_bl = costes[idx_mejor_actual]

            print(f"   >>> Ejecutando BT (Hito {hito_actual}, Prof {profundidad}) en eval {evaluaciones}...")

            # Ejecutar BT Modificada
            perm_mejorado, coste_mejorado = busqueda_tabu_modificada(
                perm_elite_bl, coste_elite_bl, profundidad, n, flujo, distancia, tenencia_tabu
            )

            # Si mejora, sustituimos
            if coste_mejorado < coste_elite_bl:
                print(f"       Mejora BT conseguida: {coste_elite_bl} -> {coste_mejorado}")
                poblacion[idx_mejor_actual] = perm_mejorado
                costes[idx_mejor_actual] = coste_mejorado

                if coste_mejorado < mejor_coste:
                    mejor_coste = coste_mejorado
                    mejor_global = perm_mejorado.copy()

            ultimo_trigger = evaluaciones

        # Actualizar mejor global normal
        idx_mejor_gen = np.argmin(costes)
        if costes[idx_mejor_gen] < mejor_coste:
            mejor_coste = costes[idx_mejor_gen]
            mejor_global = poblacion[idx_mejor_gen].copy()

    fin = datetime.now()
    duracion = fin - inicio

    print(f"--- RESULTADOS MGEN ---")
    print(f"Dataset: {ruta}")
    print(f"Mejor coste encontrado: {mejor_coste}")
    print(f"Tiempo total: {duracion}")

    escritura_logs(
        f"log_mgen_{semilla}.txt",
        f"Mejor coste MGEN: {mejor_coste}",
        duracion,
        f"Algoritmo: MGEN - Dataset: {ruta} - Semilla: {semilla} - OX2 0.7 - E=1 - kBest=2 - BT triggers 1/2/5k",
        tamano=n
    )

    return mejor_global, mejor_coste, duracion

## Inicio del programa

In [9]:
parametros = leer_parametros("parametros.txt")
algoritmo = parametros.get("algoritmo","")
dataset = parametros.get("dataset", "")
semilla = int(parametros.get("seed",0))

if algoritmo == "greedy":
  print("Greedy normal")
  greedy1(dataset)
elif algoritmo == "greedyal":
  print("Greedy Aleatorio")
  greedy_aleatorio(dataset,algoritmo,semilla)
elif algoritmo == "algen":
  print("Algoritmo Evolutivo Generacional")
  perm, cost, duracion = algoritmo_gen(dataset,"parametros.txt")
elif algoritmo == "tabu":
  print("Busqueda tabu modificada")
  perm, cost, duracion = busqueda_tabu_modificada(dataset,"parametros.txt")
elif algoritmo == "memgen":
    print("Memetico Generacional")
    perm, cost, duracion= algoritmo_memetico_gen(dataset,"parametros.txt")
else:
  print("Algoritmo no reconocido")

Memetico Generacional
Permutación generada (unidad -> localización): [27  1  8 14 24  3 26  0 29 16 18  4 22 15  6 13 11 20 10 19 12 28 23  2
 25 21  5  7  9 17]
Coste total (GRA): 193840
Permutación generada (unidad -> localización): [27  1  8 14 24  3 26  0 29 16 18  4 22 15  6 13 11 20 10 19 12 28 23  2
 25 21  5  7  9 17]
Coste total (GRA): 193840
Permutación generada (unidad -> localización): [27  1  8 14 24  3 26  0 29 16 18  4 22 15  6 13 11 20 10 19 12 28 23  2
 25 21  5  7  9 17]
Coste total (GRA): 193840
Permutación generada (unidad -> localización): [27  1  8 14 24  3 26  0 29 16 18  4 22 15  6 13 11 20 10 19 12 28 23  2
 25 21  5  7  9 17]
Coste total (GRA): 193840
Permutación generada (unidad -> localización): [27  1  8 14 24  3 26  0 29 16 18  4 22 15  6 13 11 20 10 19 12 28 23  2
 25 21  5  7  9 17]
Coste total (GRA): 193840
Permutación generada (unidad -> localización): [27  1  8 14 24  3 26  0 29 16 18  4 22 15  6 13 11 20 10 19 12 28 23  2
 25 21  5  7  9 17]
Coste to